In [1]:
import os
import pandas as pd
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from io import StringIO

In [ ]:
gauth = GoogleAuth()
gauth.settings['client_config_backend'] = 'service'
gauth.settings['service_config'] = {
    'client_json_file_path': 'service_account.json',
    'client_user_email': 'sheet-accessor-bot@sheet-accessor-468220.iam.gserviceaccount.com'
}
gauth.ServiceAuth()
drive = GoogleDrive(gauth)

In [3]:
folder_ids = [
    '1kg-I2DYN4HsQ-jIFhym7nmvkubc9GyFD',
    '1tUormlmSj0P8342ZVdkFHwK5Zzlz4pd_',
    '1uNGRy3a3EBJjnjlppZ29cLqlvTxDijG7',
    '1icyrHfA04vVziPAf4B8cL8x6pyltLl4m',
    '1W4k8-Y1GWLJv74j8mXCKJPxaIngui4xP',
    '1LEYANh7PIV9BIC4GpmWByWR4mi2dNtih',
    '1FbOjmMtKKbgpDou-eyVbhb162RQJMNy-',
    '10-NQy1b9lXuXnm5X_FuzYpetI_1PgjOQ',
    '1IYkTrGI07fTcjaggY_12V72syNATNSIp',
    '1gi-k4cgD-XR_AiaSq6J9OPf3OWwkIPCt',
    '145ECofup4M7ZnA1dUF5_pDZN3OWEK1Yb',
    '1vsjVWIWOAH1xN8RPvhgmI6ky1vA7CfcJ'
]

In [4]:
os.makedirs('extracted', exist_ok = True)
os.makedirs('processed', exist_ok = True)

In [ ]:
def preprocess(file_path):
    df = pd.read_csv(file_path, encoding='utf-8-sig')

    df.columns = df.columns.str.strip()
    df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
    df = df.dropna(how='all')

    if df.columns[0] != 'instID':
        df.rename(columns={df.columns[0]: 'instID'}, inplace=True)

    if 'year' not in df.columns or df['year'].isna().all():
        file_name = os.path.basename(file_path)
        inferred_year = ''.join(filter(str.isdigit, file_name))
        df['year'] = inferred_year if inferred_year else ''
    else:
        year = df['year'].dropna().iloc[0]
        df['year'] = year

    expected_columns = [
        'instID', 'type', 'instName', 'language', 'instNote', 'placeName', 'region',
        'county_orig', 'county_reg', 'city_orig', 'city_reg', 'state_orig', 'state_reg',
        'latitude', 'longitude', 'attendingInstID', 'attendingPersID', 'attendingChurch',
        'attendingChurchFrequency', 'attendingChurchNote', 'persID', 'persTitle', 'persName',
        'persSuffix', 'persRole', 'persNote', 'memberType', 'member', 'affiliated',
        'year', 'diocese', 'order'
    ]
    for col in expected_columns:
        if col not in df.columns:
            df[col] = ''

    df = df.apply(lambda col: col.map(lambda x: x.encode('utf-8-sig', errors='ignore').decode('utf-8') if isinstance(x, str) else x))

    df.loc[(df['instID'].notna()) & (df['instName'].isna()), 'instName'] = 'not recorded'

    primary_cols = ['instID', 'type', 'instName', 'latitude', 'longitude']
    df[primary_cols] = df[primary_cols].ffill()
    
    df.rename(columns={'type':'church_type'}, inplace = True)
    
    group_cols = [
        'language', 'instNote', 'placeName', 'region', 'county_orig', 'county_reg',
        'city_orig', 'state_orig', 'city_reg', 'memberType', 'member', 'affiliated'
    ]
    df[group_cols] = df.groupby('instID')[group_cols].ffill()

    df['language'] = df['language'].fillna('English')

    if df['year'].notna().any():
        year = df['year'].dropna().iloc[0]
        df['year'] = year
    
    file_name = os.path.basename(file_path)
    place_name = file_name.split('1')[0]

    if 'diocese' in df.columns and df['diocese'].notna().any():
        diocese = df['diocese'].dropna().iloc[0]
    else:
        diocese = None

    df['diocese_orig'] = diocese if diocese else place_name
    df['diocese_reg'] = place_name

    df.drop(columns=['diocese'], inplace=True, errors='ignore')
    df.rename(columns={'diocese_orig': 'diocese'}, inplace=True)

    return df

In [8]:
for folder_id in folder_ids:
    file_list = drive.ListFile({
        'q': f"'{folder_id}' in parents and trashed=false and mimeType='application/vnd.google-apps.spreadsheet'"
    }).GetList()
    for file in file_list:
        title = file['title']
        if ' ' in title:
            continue
        content = file.GetContentString(mimetype='text/csv')
        try: 
            file_name = title + '.csv'
            file_path = os.path.join("extracted", file_name)

            # Save raw CSV
            with open(file_path, 'w', encoding='utf-8-sig') as f:
                f.write(content)
                
            df_clean = preprocess(file_path)
            output_path = os.path.join("processed", file_name)
            df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(title)
        except Exception as e:
            print(f"Failed to export {title}: {e}")